In [ ]:
# architecture 1 taken from paper
# maybe later add more architectures
hidden_layer_sizes = [20, 10, 5]
dropout_rates_train = [0, 0, 0, 0]
activation_functions = ["relu", "relu", "relu", "none"]

In [ ]:
# propensity scores taken from paper

hidden_layer_sizes_treatment = [50, 30]
activation_functions_treatment = ["relu", "relu", "none"]
dropout_rates_train_treatment = [0, 0, 0]
dropout_rates_test_treatment = [0 for i in dropout_rates_train_treatment]

In [ ]:
# hyperparameters

train_proportion = 0.9
max_nepochs = 5000
max_epochs_without_change = 30
optimizer = "Adam"
learning_rate = 0.009
batch_size = 128
batch_size_t = None

In [ ]:
# Regularization parameters
alpha = 0.0
r = 0.2

In [ ]:
import pandas as pd

# Load the real data
data = pd.read_csv(
    "bld/difference_in_difference_with_deep_learning/data/cleaned_data.csv",
)  # Change the filename to your actual data file

# Assuming 'wage_year' is the variable of interest
X = data[["Individual", "Age", "WagePartner"]]  # Features
T_real = data["interaction"]  # Treatment assignment
Y = data["wage_year"]  # Outcome

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming you have your data stored in variables X, T_real, and Y
# and train_proportion is set to 0.9

# Calculate test size based on train_proportion
test_size = 1.0 - train_proportion

# Set up the test and training split
X_train, X_valid, T_train, T_valid, Y_train, Y_valid = train_test_split(
    X,
    T_real,
    Y,
    test_size=test_size,
    random_state=42,
)

# Print the shapes of the training and validation sets to verify
print("X_train shape:", X_train.shape)
print("X_valid shape:", X_valid.shape)
print("T_train shape:", T_train.shape)
print("T_valid shape:", T_valid.shape)
print("Y_train shape:", Y_train.shape)
print("Y_valid shape:", Y_valid.shape)

from fakedata get some values
- mu0_real
- tau_real
- T_real
- seed
- prob_of_T
- tau_true_mean